In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import tensorflow as tf

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'Phishing_Legitimate_full.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
phishing = pd.read_csv(file_content_stream)
phishing.head()

,id,NumDots,SubdomainLevel,PathLevel,UrlLength,NumDash,NumDashInHostname,AtSymbol,TildeSymbol,NumUnderscore,NumPercent,NumQueryComponents,NumAmpersand,NumHash,NumNumericChars,NoHttps,RandomString,IpAddress,DomainInSubdomains,DomainInPaths,HttpsInHostname,HostnameLength,PathLength,QueryLength,DoubleSlashInPath,NumSensitiveWords,EmbeddedBrandName,PctExtHyperlinks,PctExtResourceUrls,ExtFavicon,InsecureForms,RelativeFormAction,ExtFormAction,AbnormalFormAction,PctNullSelfRedirectHyperlinks,FrequentDomainNameMismatch,FakeLinkInStatusBar,RightClickDisabled,PopUpWindow,SubmitInfoToEmail,IframeOrFrame,MissingTitle,ImagesOnlyInForm,SubdomainLevelRT,UrlLengthRT,PctExtResourceUrlsRT,AbnormalExtFormActionR,ExtMetaScriptLinkRT,PctExtNullSelfRedirectHyperlinksRT,CLASS_LABEL
0,1,3,1,5,72,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,21,44,0,0,0,0,0.000,0.250000,1,1,0,0,0,0.0,0,0,0,0,0,0,0,1,1,0,1,1,-1,1,1
1,2,3,1,3,144,0,0,0,0,2,0,2,1,0,41,1,0,0,0,0,0,17,16,103,0,1,0,0.000,0.000000,0,1,0,0,0,0.0,0,0,0,0,0,0,0,0,1,-1,1,1,1,1,1
2,3,3,1,2,58,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,27,24,0,0,0,0,0.375,1.000000,1,1,0,0,0,0.0,0,0,0,0,0,0,0,0,1,0,-1,1,-1,0,1
3,4,3,1,6,79,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,22,50,0,0,0,1,1.000,0.095238,1,1,0,0,0,0.0,1,0,0,0,1,0,0,0,1,-1,1,1,1,-1,1
4,5,3,0,4,46,0,0,0,0,0,0,0,0,0,2,1,1,0,0,1,0,10,29,0,0,0,0,1.000,1.000000,0,0,0,1,0,0.0,1,0,0,0,0,1,0,0,1,1,-1,0,-1,-1,1


In [6]:
# dropping id
phishing = phishing.drop(columns = 'id', axis = 1)

In [ ]:
# we have a lot of input variables, our first step will be to find the most importaint variables through variable selection 

In [42]:
# defining input and target variables 
x = phishing.drop(columns = 'CLASS_LABEL', axis = 1)
y = phishing['CLASS_LABEL']

# list to store coefficents 
coef = list()

for i in range(0, 100): 
    # splitting the data into train and test 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)
    
    # running LASSO cross-validation to estimate optimal lambda
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(x_train, y_train)
    
    # building LASSO regression with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(x_train, y_train)
    
    # storing estimated coefficients
    coef.append(lasso_md.coef_)
    
# putting coefficient list in data-frame
coef_data = pd.DataFrame(coef)

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.16587833057354828, tolerance: 0.15999943749999962
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4885872811739773, tolerance: 0.15999943749999962
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:633: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9549667683638745, tolerance: 0.15999943749999962
  model = cd_fast.enet_coordinate_desce

In [43]:
coef_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,0.036591,-0.079106,0.028937,0.0,-0.025869,0.049222,-0.352043,0.100027,-0.010911,-0.040916,-0.083966,0.034729,-0.330458,0.000057,-0.364419,-0.022677,-0.090615,0.064131,-0.056819,0.0,0.006070,0.002850,0.003053,0.175424,0.084572,-0.031763,-0.525887,-0.158282,0.148433,0.292312,-0.077082,-0.064465,-0.035534,-0.336000,0.379459,-0.071584,0.080897,-0.039994,-0.162671,-0.143882,0.232410,-0.021658,-0.016487,0.075309,0.005548,0.029416,0.049030,-0.338055
1,0.037590,-0.079750,0.030375,0.0,-0.025396,0.051722,-0.645375,0.104905,-0.010642,-0.042362,-0.074824,0.028079,-0.273882,-0.000225,-0.370821,-0.024421,-0.112762,0.054873,-0.052038,0.0,0.006147,0.002763,0.002839,0.160311,0.083979,-0.022095,-0.556776,-0.178075,0.152674,0.293818,-0.080160,-0.089530,-0.086076,-0.347354,0.390248,-0.077143,0.089652,-0.058213,-0.164444,-0.142328,0.236158,-0.026299,-0.004161,0.072669,0.007887,0.000000,0.043422,-0.343935
2,0.038477,-0.084781,0.030335,0.0,-0.025232,0.052090,-0.538094,0.107667,-0.013598,-0.037073,-0.080489,0.035681,-0.349838,0.000238,-0.363784,-0.019520,-0.112504,0.054062,-0.061523,0.0,0.006911,0.002617,0.002714,0.029637,0.084501,-0.032762,-0.489412,-0.171579,0.149088,0.287750,-0.076253,-0.086537,-0.073342,-0.305003,0.395190,-0.090273,0.047178,-0.046393,-0.162385,-0.133274,0.214981,-0.017547,-0.005322,0.074304,0.007833,0.008843,0.053751,-0.318307
3,0.037616,-0.079604,0.030258,0.0,-0.026423,0.053747,-0.538557,0.080077,-0.013500,-0.038148,-0.077017,0.028517,-0.320077,0.000054,-0.343954,-0.014926,-0.113053,0.069203,-0.045757,0.0,0.006071,0.002833,0.003035,0.117949,0.086836,-0.045686,-0.481908,-0.160565,0.149549,0.283810,-0.075245,-0.085044,-0.093838,-0.293455,0.396889,-0.078980,0.063430,-0.065227,-0.164127,-0.141907,0.236889,-0.023185,-0.000000,0.074619,0.008297,0.000000,0.052536,-0.317802
4,0.035854,-0.081097,0.029925,0.0,-0.027390,0.055121,-0.456906,0.081219,-0.014212,-0.040544,-0.075176,0.027090,-0.269192,-0.000140,-0.355458,-0.019325,-0.101338,0.043454,-0.053072,0.0,0.006537,0.003011,0.002933,0.206185,0.088811,-0.031439,-0.499912,-0.166554,0.144332,0.287951,-0.078609,-0.089046,-0.086831,-0.314746,0.383554,-0.049854,0.072730,-0.079822,-0.161006,-0.134648,0.228298,-0.015199,-0.014403,0.075350,0.008397,0.000000,0.050363,-0.326197


In [44]:
totals = list()

for i in range(0,48):
    totals.append(sum(coef_data[i] == 0.0))
    
total_zeros = pd.DataFrame(totals)
total_zeros.index = [phishing.drop(columns = 'CLASS_LABEL', axis = 1)]
total_zeros = total_zeros.sort_values(by = 0, ascending = False)
total_zeros

,0
UrlLength,100
HttpsInHostname,100
AbnormalExtFormActionR,31
SubdomainLevelRT,31
NumNumericChars,17
AtSymbol,2
PopUpWindow,1
FakeLinkInStatusBar,0
ExtFavicon,0
InsecureForms,0


In [47]:
phishing_importances = list()

for i in range (1,100):
    # splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

    # building RF model
    phishing_md = RandomForestClassifier(n_estimators = 500).fit(x_train, y_train)

    #extracting importaint features 
    phishing_importances.append(phishing_md.feature_importances_)
    
phishing_results = pd.DataFrame(phishing_importances)
phishing_results.columns = [phishing.drop(columns = 'CLASS_LABEL', axis = 1)] 

avg_importances = pd.DataFrame({'Importance': np.mean(phishing_results)})
avg_importances = avg_importances.sort_values(by = 'Importance', ascending = False)
avg_importances

,Importance
PctExtHyperlinks,0.189256
PctExtNullSelfRedirectHyperlinksRT,0.159989
PctExtResourceUrls,0.081720
FrequentDomainNameMismatch,0.081641
PctNullSelfRedirectHyperlinks,0.065248
NumDash,0.047650
NumNumericChars,0.036920
ExtMetaScriptLinkRT,0.035939
InsecureForms,0.031211
SubmitInfoToEmail,0.028372


In [48]:
avg_importances['Num Zeros'] = total_zeros[0]
avg_importances

,Importance,Num Zeros
PctExtHyperlinks,0.189256,0
PctExtNullSelfRedirectHyperlinksRT,0.159989,0
PctExtResourceUrls,0.081720,0
FrequentDomainNameMismatch,0.081641,0
PctNullSelfRedirectHyperlinks,0.065248,0
NumDash,0.047650,0
NumNumericChars,0.036920,17
ExtMetaScriptLinkRT,0.035939,0
InsecureForms,0.031211,0
SubmitInfoToEmail,0.028372,0


Logistic Regression

In [3]:
# defining input and target variables 
x = phishing[['PctExtHyperlinks', 'PctExtNullSelfRedirectHyperlinksRT', 'FrequentDomainNameMismatch', 'PctExtResourceUrls', 'PctNullSelfRedirectHyperlinks', 'NumDash']]
y = phishing['CLASS_LABEL']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

# building logistic regression model
logit_md = LogisticRegression().fit(x_train, y_train)

# predicting on the test data
logit_pred = logit_md.predict_proba(x_test)[:,1]
# changing likelyhoods to labels
logit_label = np.where(logit_pred < .25, 0, 1)
# computing the accuracy and recall
print(accuracy_score(y_test, logit_label))
print(recall_score(y_test, logit_label))

0.779
0.9539748953974896


Neural Network

In [20]:
# scaling the data
scaler = MinMaxScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_train.columns)

nn_recall = list()
nn_accuracy = list()

for i in range(0,5):
    # defining the first MLP model 
    md1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_dim = 6, activation = 'relu'),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(2, activation = 'sigmoid')
    ])
    md1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    md1.fit(x_train, tf.keras.utils.to_categorical(y_train, num_classes = 2), epochs = 100, batch_size = 500,
            validation_data = (x_test, tf.keras.utils.to_categorical(y_test, num_classes = 2)))
    # predicting on test data
    md1_pred = md1.predict(x_test)[:,1]
    #changing likilyhood to label 
    md1_label = np.where(md1_pred < .50, 0, 1)
    # computing the accuracy and recall
    nn_accuracy.append(accuracy_score(y_test, md1_label))
    nn_recall.append(recall_score(y_test, md1_label))

print('avg recall', np.mean(nn_recall))
print('avg accuracy', np.mean(nn_accuracy))

Epoch 1/100
16/16 [==============================] - 1s 15ms/step - loss: 0.8780 - accuracy: 0.5027 - val_loss: 0.8287 - val_accuracy: 0.4750
Epoch 2/100
16/16 [==============================] - 0s 13ms/step - loss: 0.7824 - accuracy: 0.5015 - val_loss: 0.7582 - val_accuracy: 0.4770
Epoch 3/100
16/16 [==============================] - 0s 4ms/step - loss: 0.7293 - accuracy: 0.5058 - val_loss: 0.7139 - val_accuracy: 0.5030
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6947 - accuracy: 0.5195 - val_loss: 0.6869 - val_accuracy: 0.5065
Epoch 5/100
16/16 [==============================] - 0s 4ms/step - loss: 0.6741 - accuracy: 0.5319 - val_loss: 0.6691 - val_accuracy: 0.5875
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6591 - accuracy: 0.6317 - val_loss: 0.6553 - val_accuracy: 0.6390
Epoch 7/100
16/16 [==============================] - 0s 4ms/step - loss: 0.6468 - accuracy: 0.6886 - val_loss: 0.6436 - val_accuracy: 0.7095
Epoch 8/100

Support Vecor Machine Classifier

In [17]:
# building the svm model
md3 = SVC(kernel = 'rbf', probability = True).fit(x_train, y_train)
# predicting on test
md3_pred = md3.predict_proba(x_test)[:,1]
# likilyhoods to labels
md3_label = np.where(md3_pred < .25, 0, 1)
# computing the accuracy and recall
print(accuracy_score(y_test, md3_label))
print(recall_score(y_test, md3_label))

# building the svm model
md3 = SVC(kernel = 'poly', probability = True).fit(x_train, y_train)
# predicting on test
md3_pred = md3.predict_proba(x_test)[:,1]
# likilyhoods to labels
md3_label = np.where(md3_pred < .25, 0, 1)
# computing the accuracy and recall
print(accuracy_score(y_test, md3_label))
print(recall_score(y_test, md3_label))

0.8445
0.948
0.698
0.992


Random Forest

In [26]:
# building RF model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(x_train, y_train)

# predicting 
rf_pred = rf_md.predict_proba(x_test)[:,1]

# changing liklihoods to labels
rf_labels = np.where(rf_pred < 0.25, 0, 1)

# computing the accuracy and recall
print(accuracy_score(y_test, rf_labels))
print(recall_score(y_test, rf_labels))

0.937
0.9756592292089249


Ada Boost

In [35]:
# Ada Boost model
# building the model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = .001).fit(x_train, y_train)

# predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#liklyhoods to labels
ada_label = np.where(ada_pred < .25, 0, 1)

# computing the accuracy and recall
print(accuracy_score(y_test, ada_label))
print(recall_score(y_test, ada_label))

0.932
0.9715686274509804


Gradient Boosting 

In [38]:
# gradient boosting
# building the model 
gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = .001).fit(x_train, y_train)

# predicting on test
gb_pred = gb_md.predict_proba(x_test)[:,1]

#liklyhoods to labels
gb_label = np.where(gb_pred < .25, 0, 1)

# computing the accuracy and recall
print(accuracy_score(y_test, gb_label))
print(recall_score(y_test, gb_label))

0.51
1.0


Parameter selection

In [7]:
# defining input and target variables
x = phishing[['PctExtHyperlinks', 'PctExtNullSelfRedirectHyperlinksRT', 'FrequentDomainNameMismatch', 'PctExtResourceUrls', 'PctNullSelfRedirectHyperlinks', 'NumDash']]
y = phishing['CLASS_LABEL']

from itertools import product
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], columns = dictionary.keys())
rf_dictionary = {'n_tree': [100, 500, 1000, 1500, 2000], 'depth': [3, 5, 7]}
rf_parameters = expand_grid(rf_dictionary)
ada_gb_dictionary = {'n_tree': [100, 500, 1000, 1500, 2000], 'depth': [3, 5, 7], 'learning_rate': [0.1, 0.01, 0.001]}
ada_gb_parameters = expand_grid(ada_gb_dictionary)
svc_parameters = ['rbf', 'poly']

rf_recall_results = pd.DataFrame()
rf_accuracy_results = pd.DataFrame()
ada_recall_results = pd.DataFrame()
ada_accuracy_results = pd.DataFrame()
gb_recall_results = pd.DataFrame()
gb_accuracy_results = pd.DataFrame()
svc_recall_results = pd.DataFrame()
svc_accuracy_results = pd.DataFrame()
logit_recall_results = pd.DataFrame()
logit_accuracy_results = pd.DataFrame()

for j in range(0,5):
    rf_recall = list()
    rf_accuracy = list()
    ada_recall = list()
    ada_accuracy = list()
    gb_recall = list()
    gb_accuracy = list()
    svc_recall = list()
    svc_accuracy = list()
    logit_recall = list()
    logit_accuracy = list()
    
    # splitting the data into train and test 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, stratify = y)
    
    print('itteration ', j)
    
    # building logistic regression model
    logit_md = LogisticRegression().fit(x_train, y_train)
    # predicting on the test data
    logit_pred = logit_md.predict_proba(x_test)[:,1]
    # changing likelyhoods to labels
    logit_label = np.where(logit_pred < .25, 0, 1)
    # computing the accuracy and recall
    logit_recall.append(recall_score(y_test, logit_label))
    logit_accuracy.append(accuracy_score(y_test, logit_label))
    logit_recall_results['RF Recall ' + str(j)] = logit_recall
    logit_accuracy_results['RF accuracy ' + str(j)] = logit_accuracy
    
    for i in range (0, len(svc_parameters)):
        # building the svm model
        svc = SVC(kernel = svc_parameters[i], probability = True).fit(x_train, y_train)
        # predicting on test
        svc_pred = svc.predict_proba(x_test)[:,1]
        # likilyhoods to labels
        svc_label = np.where(svc_pred < .25, 0, 1)
        # computing the accuracy and recall
        svc_recall.append(recall_score(y_test, svc_label))
        svc_accuracy.append(accuracy_score(y_test, svc_label))
    svc_recall_results['RF Recall ' + str(j)] = svc_recall
    svc_accuracy_results['RF accuracy ' + str(j)] = svc_accuracy

    for i in range(0, len(rf_parameters)):
        # RF model
        rf_md = RandomForestClassifier(n_estimators = rf_parameters['n_tree'][i], max_depth = rf_parameters['depth'][i]).fit(x_train, y_train)
        # predicting on test
        rf_pred = rf_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        rf_label = np.where(rf_pred < .1, 0, 1)
        # computing recall and accuracy
        rf_recall.append(recall_score(y_test, rf_label))
        rf_accuracy.append(accuracy_score(y_test, rf_label))
    rf_recall_results['RF Recall ' + str(j)] = rf_recall
    rf_accuracy_results['RF accuracy ' + str(j)] = rf_accuracy
    
    for i in range(0, len(ada_gb_parameters)):
        # Ada Boost model
        ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = ada_gb_parameters['depth'][i]), n_estimators = ada_gb_parameters['n_tree'][i], learning_rate = ada_gb_parameters['learning_rate'][i]).fit(x_train, y_train)
        # predicting on test
        ada_pred = ada_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        ada_label = np.where(ada_pred < .1, 0, 1)
        # computing recall and accuracy
        ada_recall.append(recall_score(y_test, ada_label))
        ada_accuracy.append(accuracy_score(y_test, ada_label))
    ada_recall_results['Ada Recall ' + str(j)] = ada_recall
    ada_accuracy_results['Ada accuracy ' + str(j)] = ada_accuracy

    for i in range(0, len(ada_gb_parameters)):   
        # gradient boosting
        gb_md = GradientBoostingClassifier(max_depth = ada_gb_parameters['depth'][i], n_estimators = ada_gb_parameters['n_tree'][i], learning_rate = ada_gb_parameters['learning_rate'][i]).fit(x_train, y_train)
        # predicting on test
        gb_pred = gb_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        gb_label = np.where(gb_pred < .1, 0, 1)
        # computing recall and accuracy
        gb_recall.append(recall_score(y_test, gb_label))
        gb_accuracy.append(accuracy_score(y_test, gb_label))
    gb_recall_results['GB Recall ' + str(j)] = gb_recall
    gb_accuracy_results['GB accuracy ' + str(j)] = gb_accuracy
    
logit_recall_results['Total AVG Recall'] = logit_recall_results.mean(axis = 1)
logit_accuracy_results['Total AVG Accuracy'] = logit_accuracy_results.mean(axis = 1)
svc_recall_results['Total AVG Recall'] = svc_recall_results.mean(axis = 1)
svc_accuracy_results['Total AVG Accuracy'] = svc_accuracy_results.mean(axis = 1)
rf_recall_results['Total AVG Recall'] = rf_recall_results.mean(axis = 1)
rf_accuracy_results['Total AVG Accuracy'] = rf_accuracy_results.mean(axis = 1)
ada_recall_results['Total AVG Recall'] = ada_recall_results.mean(axis = 1)
ada_accuracy_results['Total AVG Accuracy'] = ada_accuracy_results.mean(axis = 1)
gb_recall_results['Total AVG Recall'] = gb_recall_results.mean(axis = 1)
gb_accuracy_results['Total AVG Accuracy'] = gb_accuracy_results.mean(axis = 1)

itteration  0


KeyboardInterrupt: 

In [ ]:
# saving results
logit_recall_results.to_csv('logit_recall_results.csv', index = False)
logit_accuracy_results.to_csv('logit_accuracy_results.csv', index = False)
svc_recall_results.to_csv('svc_recall_results.csv', index = False)
svc_accuracy_results.to_csv('svc_accuracy_results.csv', index = False)
rf_recall_results.to_csv('rf_recall_results.csv', index = False)
rf_accuracy_results.to_csv('rf_accuracy_results.csv', index = False)
ada_recall_results.to_csv('ada_recall_results.csv', index = False)
ada_accuracy_results.to_csv('ada_accuracy_results.csv', index = False)
gb_recall_results.to_csv('gb_recall_results.csv', index = False)
gb_accuracy_results.to_csv('gb_accuracy_results.csv', index = False)

In [8]:
logit_recall_results = pd.read_csv('logit_recall_results.csv')
logit_accuracy_results = pd.read_csv('logit_accuracy_results.csv')
svc_recall_results = pd.read_csv('svc_recall_results.csv')
svc_accuracy_results = pd.read_csv('svc_accuracy_results.csv')
rf_recall_results = pd.read_csv('rf_recall_results.csv')
rf_accuracy_results = pd.read_csv('rf_accuracy_results.csv')
ada_recall_results = pd.read_csv('ada_recall_results.csv')
ada_accuracy_results = pd.read_csv('ada_accuracy_results.csv')
gb_recall_results = pd.read_csv('gb_recall_results.csv')
gb_accuracy_results = pd.read_csv('gb_accuracy_results.csv')

In [9]:
logit_average_results = pd.DataFrame()
logit_average_results['Average Recall'] = logit_recall_results['Total AVG Recall']
logit_average_results['Average Accuracy'] = logit_accuracy_results['Total AVG Accuracy']
logit_average_results = logit_average_results.sort_values(by = 'Average Accuracy', ascending = False)
logit_average_results

,Average Recall,Average Accuracy
0,0.957,0.7915


In [10]:
svc_average_results = pd.DataFrame()
svc_average_results['Average Recall'] = svc_recall_results['Total AVG Recall']
svc_average_results['Average Accuracy'] = svc_accuracy_results['Total AVG Accuracy']
svc_average_results = svc_average_results.sort_values(by = 'Average Accuracy', ascending = False)
svc_average_results

,Average Recall,Average Accuracy
0,0.9486,0.8374
1,0.9904,0.6999


In [11]:
rf_average_results = rf_parameters
rf_average_results['Average Recall'] = rf_recall_results['Total AVG Recall']
rf_average_results['Average Accuracy'] = rf_accuracy_results['Total AVG Accuracy']
rf_average_results = rf_average_results.sort_values(by = 'Average Accuracy', ascending = False)
rf_average_results

,n_tree,depth,Average Recall,Average Accuracy
5,500,7,0.9890,0.9290
11,1500,7,0.9890,0.9284
14,2000,7,0.9890,0.9279
8,1000,7,0.9890,0.9277
2,100,7,0.9890,0.9275
1,100,5,0.9878,0.9075
10,1500,5,0.9880,0.9065
4,500,5,0.9880,0.9055
7,1000,5,0.9878,0.9051
13,2000,5,0.9878,0.9045


In [14]:
ada_average_results = ada_gb_parameters
ada_average_results['Average Recall'] = ada_recall_results['Total AVG Recall']
ada_average_results['Average Accuracy'] = ada_accuracy_results['Total AVG Accuracy']
ada_average_results = ada_average_results.sort_values(by = 'Average Accuracy', ascending = False)
ada_average_results

,n_tree,depth,learning_rate,Average Recall,Average Accuracy
7,100,7,0.010,0.9850,0.9565
17,500,7,0.001,0.9814,0.9539
16,500,7,0.010,0.9878,0.9538
26,1000,7,0.001,0.9840,0.9538
6,100,7,0.100,0.9886,0.9534
44,2000,7,0.001,0.9874,0.9515
25,1000,7,0.010,0.9876,0.9508
35,1500,7,0.001,0.9868,0.9497
8,100,7,0.001,0.9830,0.9493
34,1500,7,0.010,0.9878,0.9492


In [13]:
gb_average_results = ada_gb_parameters
gb_average_results['Average Recall'] = gb_recall_results['Total AVG Recall']
gb_average_results['Average Accuracy'] = gb_accuracy_results['Total AVG Accuracy']
gb_average_results = gb_average_results.sort_values(by = 'Average Accuracy', ascending = False)
gb_average_results

,n_tree,depth,learning_rate,Average Recall,Average Accuracy
42,2000,7,0.100,0.9838,0.9638
33,1500,7,0.100,0.9838,0.9634
24,1000,7,0.100,0.9850,0.9632
30,1500,5,0.100,0.9864,0.9609
15,500,7,0.100,0.9864,0.9609
39,2000,5,0.100,0.9852,0.9608
21,1000,5,0.100,0.9874,0.9592
12,500,5,0.100,0.9916,0.9570
43,2000,7,0.010,0.9900,0.9565
34,1500,7,0.010,0.9916,0.9549


In [25]:
# best models. 
# logit
# svc = rbf
# rf = n_estimators = 500, max_depth = 7
# ada = n_estimators = 100, max_depth = 7, learning_rate = .01
# gb = n_estimators = 2000, max_depth = 7, learning_rate = .1

# defining input and target variables
x = phishing[['PctExtHyperlinks', 'PctExtNullSelfRedirectHyperlinksRT', 'FrequentDomainNameMismatch', 'PctExtResourceUrls', 'PctNullSelfRedirectHyperlinks', 'NumDash']]
y = phishing['CLASS_LABEL']

# splitting the data into train and test 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, stratify = y)

# building logistic regression model
logit_md = LogisticRegression().fit(x_train, y_train)
# predicting on the test data
logit_pred = logit_md.predict_proba(x_test)[:,1]
# changing likelyhoods to labels
logit_label = np.where(logit_pred < .1, 0, 1)
# computing the accuracy and recall
print('logit recall:', recall_score(y_test, logit_label))
print('logit accuracy:', accuracy_score(y_test, logit_label))

# building the svm model
svc = SVC(kernel = 'rbf', probability = True).fit(x_train, y_train)
# predicting on test
svc_pred = svc.predict_proba(x_test)[:,1]
# likilyhoods to labels
svc_label = np.where(svc_pred < .1, 0, 1)
# computing the accuracy and recall
print('svc recall:', recall_score(y_test, svc_label))
print('svc accuracy:', accuracy_score(y_test, svc_label))

rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(x_train, y_train)
# predicting on test
rf_pred = rf_md.predict_proba(x_test)[:,1]
#liklyhoods to labels
rf_label = np.where(rf_pred < .1, 0, 1)
# computing recall and accuracy
print('rf recall:', recall_score(y_test, rf_label))
print('rf accuracy:',accuracy_score(y_test, rf_label))
    
# Ada Boost model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 7), n_estimators = 100, learning_rate = .01).fit(x_train, y_train)
# predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]
#liklyhoods to labels
ada_label = np.where(ada_pred < .1, 0, 1)
# computing recall and accuracy
print('ada recall:', recall_score(y_test, ada_label))
print('ada accuracy:', accuracy_score(y_test, ada_label))
 
# gradient boosting
gb_md = GradientBoostingClassifier(max_depth = 7, n_estimators = 2000, learning_rate = .1).fit(x_train, y_train)
# predicting on test
gb_pred = gb_md.predict_proba(x_test)[:,1]
#liklyhoods to labels
gb_label = np.where(gb_pred < .1, 0, 1)
# computing recall and accuracy
print('gb recall:', recall_score(y_test, gb_label))
print('gb accuracy:', accuracy_score(y_test, gb_label))

logit recall: 0.953
logit accuracy: 0.794
svc recall: 0.943
svc accuracy: 0.842
rf recall: 0.988
rf accuracy: 0.923
ada recall: 0.981
ada accuracy: 0.953
gb recall: 0.981
gb accuracy: 0.96


In [7]:
# optimising cutoff values
# cutoffs to test [.1, .15, .20, .25, .30, .35, .40, .45, .50, .55]
# defining input and target variables
x = phishing[['PctExtHyperlinks', 'PctExtNullSelfRedirectHyperlinksRT', 'FrequentDomainNameMismatch', 'PctExtResourceUrls', 'PctNullSelfRedirectHyperlinks', 'NumDash']]
y = phishing['CLASS_LABEL']

cutoffs = [.05, .1, .15, .20, .25, .30, .35, .40, .45, .50, .55]

rf_recall_cutoff_results = pd.DataFrame()
rf_accuracy_cutoff_results = pd.DataFrame()
ada_recall_cutoff_results = pd.DataFrame()
ada_accuracy_cutoff_results = pd.DataFrame()
gb_recall_cutoff_results = pd.DataFrame()
gb_accuracy_cutoff_results = pd.DataFrame()

for j in range(0,10):
    rf_cutoff_recall = list()
    rf_cutoff_accuracy = list()
    ada_cutoff_recall = list()
    ada_cutoff_accuracy = list()
    gb_cutoff_recall = list()
    gb_cutoff_accuracy = list()
    
    # splitting the data into train and test 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, stratify = y)
    
    print('itteration ', j+1)

    for i in range(0, len(cutoffs)):
        # RF model
        rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(x_train, y_train)
        # predicting on test
        rf_pred = rf_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        rf_label = np.where(rf_pred < cutoffs[i], 0, 1)
        # computing recall and accuracy
        rf_cutoff_recall.append(recall_score(y_test, rf_label))
        rf_cutoff_accuracy.append(accuracy_score(y_test, rf_label))
    rf_recall_cutoff_results['RF Recall ' + str(j)] = rf_cutoff_recall
    rf_accuracy_cutoff_results['RF accuracy ' + str(j)] = rf_cutoff_accuracy
    
    for i in range(0, len(cutoffs)):
        # Ada Boost model
        ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 7), n_estimators = 100, learning_rate = .01).fit(x_train, y_train)
        # predicting on test
        ada_pred = ada_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        ada_label = np.where(ada_pred < cutoffs[i], 0, 1)
        # computing recall and accuracy
        ada_cutoff_recall.append(recall_score(y_test, ada_label))
        ada_cutoff_accuracy.append(accuracy_score(y_test, ada_label))
    ada_recall_cutoff_results['Ada Recall ' + str(j)] = ada_cutoff_recall
    ada_accuracy_cutoff_results['Ada accuracy ' + str(j)] = ada_cutoff_accuracy

    for i in range(0, len(cutoffs)):   
        # gradient boosting
        gb_md = GradientBoostingClassifier(max_depth = 7, n_estimators = 2000, learning_rate = .1).fit(x_train, y_train)
        # predicting on test
        gb_pred = gb_md.predict_proba(x_test)[:,1]
        #liklyhoods to labels
        gb_label = np.where(gb_pred < cutoffs[i], 0, 1)
        # computing recall and accuracy
        gb_cutoff_recall.append(recall_score(y_test, gb_label))
        gb_cutoff_accuracy.append(accuracy_score(y_test, gb_label))
    gb_recall_cutoff_results['GB Recall ' + str(j)] = gb_cutoff_recall
    gb_accuracy_cutoff_results['GB accuracy ' + str(j)] = gb_cutoff_accuracy
    
rf_recall_cutoff_results['Total AVG Recall'] = rf_recall_cutoff_results.mean(axis = 1)
rf_accuracy_cutoff_results['Total AVG Accuracy'] = rf_accuracy_cutoff_results.mean(axis = 1)
ada_recall_cutoff_results['Total AVG Recall'] = ada_recall_cutoff_results.mean(axis = 1)
ada_accuracy_cutoff_results['Total AVG Accuracy'] = ada_accuracy_cutoff_results.mean(axis = 1)
gb_recall_cutoff_results['Total AVG Recall'] = gb_recall_cutoff_results.mean(axis = 1)
gb_accuracy_cutoff_results['Total AVG Accuracy'] = gb_accuracy_cutoff_results.mean(axis = 1)

itteration  1
itteration  2
itteration  3
itteration  4
itteration  5
itteration  6
itteration  7
itteration  8
itteration  9
itteration  10


In [11]:
cutoffs = pd.DataFrame({'values': [.05, .1, .15, .20, .25, .30, .35, .40, .45, .50, .55]})
rf_cutoff_results = cutoffs
rf_cutoff_results['Average Recall'] = rf_recall_cutoff_results['Total AVG Recall']
rf_cutoff_results['Average Accuracy'] = rf_accuracy_cutoff_results['Total AVG Accuracy']
rf_cutoff_results = rf_cutoff_results.sort_values(by = 'Average Recall', ascending = False)
rf_cutoff_results

,values,Average Recall,Average Accuracy
0,0.05,0.9945,0.86930
1,0.10,0.9896,0.92575
2,0.15,0.9859,0.93200
3,0.20,0.9829,0.93930
4,0.25,0.9814,0.94320
5,0.30,0.9786,0.94505
6,0.35,0.9758,0.94565
7,0.40,0.9738,0.94680
8,0.45,0.9723,0.94765
9,0.50,0.9700,0.94935


In [12]:
ada_cutoff_results = cutoffs
ada_cutoff_results['Average Recall'] = ada_recall_cutoff_results['Total AVG Recall']
ada_cutoff_results['Average Accuracy'] = ada_accuracy_cutoff_results['Total AVG Accuracy']
ada_cutoff_results = ada_cutoff_results.sort_values(by = 'Average Recall', ascending = False)
ada_cutoff_results

,values,Average Recall,Average Accuracy
0,0.05,0.9873,0.86470
1,0.10,0.9832,0.95040
2,0.15,0.9815,0.95620
3,0.20,0.9810,0.95690
4,0.25,0.9801,0.95760
5,0.30,0.9793,0.95795
6,0.35,0.9787,0.95835
7,0.40,0.9782,0.95880
8,0.45,0.9775,0.95935
9,0.50,0.9751,0.95905


In [13]:
gb_cutoff_results = cutoffs
gb_cutoff_results['Average Recall'] = gb_recall_cutoff_results['Total AVG Recall']
gb_cutoff_results['Average Accuracy'] = gb_accuracy_cutoff_results['Total AVG Accuracy']
gb_cutoff_results = gb_cutoff_results.sort_values(by = 'Average Recall', ascending = False)
gb_cutoff_results

,values,Average Recall,Average Accuracy
0,0.05,0.9826,0.96125
1,0.10,0.9818,0.96200
2,0.15,0.9815,0.96235
3,0.20,0.9813,0.96260
4,0.25,0.9809,0.96290
5,0.30,0.9809,0.96305
6,0.35,0.9800,0.96245
8,0.45,0.9800,0.96370
7,0.40,0.9798,0.96305
9,0.50,0.9787,0.96340


In [28]:
agregate_recall = list()
agregate_accuracy = list()

for i in range(0,100):
    print('itteration', i)
    # splitting the data into train and test 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, stratify = y)
    
    # ensamble voting classifier for weighted average predictions
    ensamble_vs = VotingClassifier(estimators=[('rf', rf_md), ('ada', ada_md), ('gb', gb_md)], voting='soft').fit(x_train, y_train)
    # predicting on test
    ensamble_pred = ensamble_vs.predict_proba(x_test)[:,1]
    # agregated likilyhoods to labels
    ensamble_label = np.where(ensamble_pred < .1, 0, 1)
    # computing recall and accuracy
    agregate_recall.append(recall_score(y_test, ensamble_label))
    agregate_accuracy.append(accuracy_score(y_test, ensamble_label))

print('Aggregated Recall:', np.mean(agregate_recall))
print('Aggregated Accuracy:', np.mean(agregate_accuracy))

itteration 0
itteration 1
itteration 2
itteration 3
itteration 4
itteration 5
itteration 6
itteration 7
itteration 8
itteration 9
itteration 10
itteration 11
itteration 12
itteration 13
itteration 14
itteration 15
itteration 16
itteration 17
itteration 18
itteration 19
itteration 20
itteration 21
itteration 22
itteration 23
itteration 24
itteration 25
itteration 26
itteration 27
itteration 28
itteration 29
itteration 30
itteration 31
itteration 32
itteration 33
itteration 34
itteration 35
itteration 36
itteration 37
itteration 38
itteration 39
itteration 40
itteration 41
itteration 42
itteration 43
itteration 44
itteration 45
itteration 46
itteration 47
itteration 48
itteration 49
itteration 50
itteration 51
itteration 52
itteration 53
itteration 54
itteration 55
itteration 56
itteration 57
itteration 58
itteration 59
itteration 60
itteration 61
itteration 62
itteration 63
itteration 64
itteration 65
itteration 66
itteration 67
itteration 68
itteration 69
itteration 70
itteration 71
it